In [22]:
import numpy as np
from itertools import combinations

In [23]:
class MarkovChain(object):
    def __init__(self, transition_matrix, states):
        """
        Initialize the MarkovChain instance.

        Parameters
        ----------
        transition_matrix: 2-D array
            A 2-D array representing the probabilities of change of
            state in the Markov Chain.

        states: 1-D array
            An array representing the states of the Markov Chain. It
            needs to be in the same order as transition_matrix.
        """
        self.transition_matrix = np.atleast_2d(transition_matrix)
        self.states = states
        self.index_dict = {self.states[index]: index for index in
                           range(len(self.states))}
        self.state_dict = {index: self.states[index] for index in
                           range(len(self.states))}

    def is_accessible(self, i_state, f_state, check_up_to_depth=10000):
        """
        Check if state f_state is accessible from i_state.

        Parameters
        ----------
        i_state: str
            The state from which the accessibility needs to be checked.

        f_state: str
            The state to which accessibility needs to be checked.
        """
    
        counter = 0
        reachable_states = [self.index_dict[i_state]]
        for state in reachable_states:
            if counter == check_up_to_depth:
                return False
            if state == self.index_dict[f_state]:
                return True
            else:
                reachable_states.extend(np.nonzero(self.transition_matrix[state, :])[0])
            counter = counter + 1
            
    def do_communicate(self, i_state, f_state):
        """
        Check if state i_state and f_state communicate with each other.
		
		Parameters
		----------
		i_state: str
		
		f_state: str
		
		"""
        if self.is_accessible(i_state, f_state) and self.is_accessible(f_state, i_state):
            return True
        return False
        
    def is_irreducible(self):
        """
        Check if the Markov Chain is irreducible.
        """
        for (i, j) in combinations(self.states, 2):
            do_communicate = self.do_communicate(i, j)
            if  do_communicate == False:
                return False
        return True

In [25]:
def find_com_pairs(states):
    """
    
    Find all communicating pairs 
    
    """
    com = []
    for (i, j) in combinations(states, 2):
        do_communicate = markov_chain.do_communicate(i, j)
        if do_communicate == True:
            com.append((i,j))
    return com

In [26]:
def com_test_def(com):    
    com_test = []
    for i in range(len(com)-1):
        i_set = set(com[i])
        for j in range(i+1, len(com)):
            j_set = set(com[j])
            if i_set & j_set:
                tup = tuple(set(com[i] + com[j]))
                if tup not in com_test:
                    com_test.append(tup)
    return com_test  

In [27]:
def find_com_classes(com, states):
    """
    Function to find communicating classes
    
    """
    com_old = com
    diff_com_test = []
    while(1):
        is_common = False
        com_new = com_test_def(com_old)

        for i in range(len(com_old)):
            for j in range(len(com_new)):
                if set(com_old[i]) & set(com_new[j]):
                    is_common = True
                if is_common == False and (com_old[i] not in diff_com_test):
                    diff_com_test.append(com_old[i]) 
            is_common = False
            
        is_common = False  
        for i in range(len(diff_com_test)):
            for j in range(len(com_old)):
                if set(diff_com_test[i]) & set(com_old[j]):
                    is_common = True
            if is_common == False and (diff_com_test[i] not in com_old):
                com_old.append(diff_com_test[i])
            is_common = False
        if len(com_new) == 0:
            
            for i in range(len(states)):
                is_common = False
                for j in range(len(com_old)):
                    if states[i] in com_old[j]:
                        is_common = True
                if is_common == False and (states[i] not in com_old):
                    com_old.append((states[i],))
            return com_old
        else:
            com_old = com_new  

In [29]:
def find_closed_classes(com_classes):
    """
    Find all closed classes and not closed classes
    
    Input: 
        communicating classes
    
    Outputs: 
        closed classes and not closed classes
    """
    not_closed_classes = []
    closed_classes = []
    is_continued = False 
    for i in range(len(com_classes)):
        for j in range(len(com_classes)):
            if i != j:
                for m in range(len(com_classes[i])):
                    for n in range(len(com_classes[j])):
                        is_accessible = markov_chain.is_accessible(com_classes[i][m], com_classes[j][n])
                        if is_accessible == True:
                            if com_classes[i] not in not_closed_classes:
                                not_closed_classes.append(com_classes[i])
                                is_continued = True
                                continue
                    if is_continued == True:
                        continue
                        
                if is_continued == True:
                    continue
        if is_continued == True:
            is_continued = False
            continue
    closed_classes = [com_cl for com_cl in com_classes if com_cl not in not_closed_classes]
    return closed_classes, not_closed_classes

In [31]:
def print_recur_trans(com_classes):
    
    """
    print all recurrent states and transient states
    
    """
    closed_classes, not_closed_classes = find_closed_classes(com_classes)
    closed_classes_combined = ()
    print("All recurrent states: ")
    for i in range(len(closed_classes)):
        closed_classes_combined = closed_classes[i] + closed_classes_combined
    closed_classes_combined = sorted(closed_classes_combined)
    print(closed_classes_combined)

    print("\nAll transient states: ")
    not_closed_classes_combined = ()
    for i in range(len(not_closed_classes)):
        not_closed_classes_combined = not_closed_classes[i] + not_closed_classes_combined
    not_closed_classes_combined = sorted(not_closed_classes_combined)
    print(not_closed_classes_combined)

In [36]:
# Test 

# states = ["0", "1", "2", "3", "4", "5"]
# transition_matrix = np.array([[0.2,0,0,0.3,0.5,0],
# [0.1,0.2,0.2,0,0.5,0],
# [0,0.1,0.3,0,0,0.6],
# [0.2,0,0,0.8,0,0],
# [0,0,0,0.4,0,0.6],
# [0,0,0,0,0.2,0.8]])

# states = ["0", "1", "2", "3", "4"]
# transition_matrix = np.array([[0.5,0.2,0.3,0,0],
# [0,0.4,0,0.6,0],
# [0.5,0,0.5,0,0],
# [0,0.5,0,0.5,0],
# [0,0.3,0,0.2,0.5]])

# states = ["0", "1", "2", "3", "4", "5"]
# transition_matrix = np.array([[0,0,0.3,0,0,0.7],
# [0,0.8,0,0,0.2,0],
# [0,0,0.8,0,0,0.2],
# [0,0.3,0.4,0.3,0,0],
# [0,0.5,0,0,0.5,0],
# [0,0,0.2,0,0,0.8]])

# states = ["0", "1", "2", "3", "4"]
# transition_matrix = np.array([[0,0,0,0,1],
# [0,0,0.3,0,0.7],
# [0.4,0.3,0.2,0.1,0],
# [0,0.6,0,0.4,0],
# [0.5,0,0,0,0.5]])

# q = 0
# states = ["0", "1", "2", "3", "4", "5", "6"]
# transition_matrix = np.array([[0.7,0,0.3,0,0,0,0],
#               [0.2,0.8,0,0,0,0,0],
#               [0,1-q,0,q,0,0,0],
#               [0,0,0,0.4,0.4,0,0.2],
#               [0,0,0,0,0.5,0.5,0],
#               [0,0,0,0.2,0,0.8,0],
#               [0,0,0,0,0,0,1]])

# states = ["0", "1", "2", "3", "4", "5"]
# transition_matrix = np.array([[1/2,1/2,0,0,0,0],
# [0,0,1,0,0,0],
# [1/3,0,0,1/3,1/3,0],
# [0,0,0,1/2,1/2,0],
# [0,0,0,0,0,1],
# [0,0,0,0,1,0]])

# states = ["0", "1", "2", "3", "4", "5", "6"]
# transition_matrix = np.array([[1/3,2/3,0,0,0,0,0],
#               [1/4,3/4,0,0,0,0,0],
#               [0,0,0,2/3,1/3,0,0],
#               [0,0,1,0,0,0,0],
#               [0,0,1,0,0,0,0],
#               [1/6,0,1/6,1/6,0,1/4,1/4],
#               [0,0,0,0,0,0,1]])


# Cau 1
states = ["0", "1", "2", "3", "4", "5"]
transition_matrix = np.array([[1/3, 0, 1/3, 0, 0, 1/3],
              [1/2, 1/4, 1/4, 0, 0, 0],
              [0, 0, 0, 0, 1, 0],
              [1/4, 1/4, 1/4, 0, 0, 1/4],
              [0, 0, 1, 0, 0, 0],
              [0, 0, 0, 0, 0, 1]])

markov_chain = MarkovChain(transition_matrix=transition_matrix,
states=states)

In [37]:
# Test: print all communicating classes

com = find_com_pairs(states)
com_classes = find_com_classes(com, states)
print(f"Number of communicating classes: {len(com_classes)}")
print(com_classes)

Number of communicating classes: 5
[('2', '4'), ('0',), ('1',), ('3',), ('5',)]


In [38]:
# Test: print all Closed classes and Not closed classes

com = find_com_pairs(states)
com_classes = find_com_classes(com, states)
closed_classes, not_closed_classes = find_closed_classes(com_classes)
print("Closed classes: ", closed_classes)    
print("Not closed classes: ", not_closed_classes)

Closed classes:  [('2', '4'), ('5',)]
Not closed classes:  [('0',), ('1',), ('3',)]


In [39]:
# Test Transience and Recurrence
com = find_com_pairs(states)
com_classes = find_com_classes(com, states)
print_recur_trans(com_classes)

All recurrent states: 
['2', '4', '5']

All transient states: 
['0', '1', '3']
